## Website
https://www.hktvmall.com/hktv/zh/homenfamily

## Environment and Imports
No need to run the cell below if you have already installed the packages

In [1]:
!pip install pyppeteer
!pip install pandas

In [2]:
from pyppeteer import launch
import asyncio
import nest_asyncio
import re
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup as bs

In [3]:
nest_asyncio.apply()
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 500)

In [4]:
def run_asyncio(func, args=None):
    return asyncio.get_event_loop().run_until_complete(func(args))

## Getting Pages

In [5]:
TABLET = 0
LAPTOP = 1
PHONE = 2
urls = [
    "https://www.hktvmall.com/hktv/zh/search?page=0&q=%3Arelevance%3Acategory%3AAA32301500001%3Azone%3Ahomenfamily%3Astreet%3Amain%3A",
    "https://www.hktvmall.com/hktv/zh/search?page=0&q=%3Arelevance%3Acategory%3AAA32300500001%3Azone%3Ahomenfamily%3Astreet%3Amain%3A",
    "https://www.hktvmall.com/hktv/zh/search?page=0&q=%3Arelevance%3Acategory%3AAA32201010001%3Azone%3Ahomenfamily%3Astreet%3Amain%3A"
]

In [6]:
# https://www.hktvmall.com/hktv/zh/search?categoryTag=AA32301500001 tablet
# https://www.hktvmall.com/hktv/zh/search?categoryTag=AA32300500001 laptop
# https://www.hktvmall.com/hktv/zh/search?categoryTag=AA32201010001 smartphone
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.max_colwidth', 50)

In [7]:
async def get_item_page(url: str):
    '''
    get the web page
    params:
        url: str, url of the web page
    return: 
        page: pyppeteer.page.Page, the web page
    '''
    browser = await launch()
    page = await browser.newPage()
    await page.setJavaScriptEnabled(enabled=True)
    await page.goto(url)
    soup = bs((await page.content()), "lxml")
    await browser.close()
    return soup

In [8]:
def get_page_num(page):
    '''
    get number of page of the current category
    params:
        page: BeautifulSoup, the bs object of the webpage
    return:
        str
    '''
    page_string = re.findall(r"/共\d+頁", str(page))
    return page_string

In [9]:
def correct_page(page):
    return (len(get_page_num(page)) > 0)

## Extracting Information

In [10]:
def handle_price(price_str):
    price_str = price_str.replace("$", '').replace(",", '')
    return float(price_str)

In [11]:
def store_customer_review(url, soup, customers):
    '''
    store the review of customers
    params:
        url: str, url of the page
        soup: BeautifulSoup, soup of the page
        customers: Dictionary, customer review to be placed in DataFrame
    '''
    ind = url.split("/")[-1]
    review_list = soup.find_all("div", {"class": "product-review-wrapper"})
    pattern = re.compile("(?<=\\xa0).*") # e.g. "評論 :\xa0Excellent"

    for review in review_list:
        name = review.find("span", {"class": "review-username"}).text
        date = review.find("span", {"class": "review-date"}).text
        rate = len(review.find_all("span", {"class": "star"}))
        content = str(review.find("div", {"class": "review-title"}).text)
        content = pattern.findall(content)[0]
        
        customers["product_index"].append(ind)
        customers["username"].append(name)
        customers["rate_date"].append(date)
        customers["user_rate"].append(rate)
        customers["buyer_comment"].append(content)
        
        print([ind, name, date, rate, content])
        

In [43]:
def extract_and_store(soup_of_page, p_type):
    '''
    extract relevant information for the product
    params:
        soup_of_page: BeautifulSoup object of the page
        p_type: str, type of product, tablet, smartphone, or laptop
    return:
        products: DataFrame in pandas that stores relevant information
    '''
    
    BASE_URL = "https://www.hktvmall.com/hktv/zh/"
    brand_product_name_list = soup_of_page.find_all("div", class_="brand-product-name")
    # https://stackoverflow.com/questions/11205386/python-beautifulsoup-get-an-attribute-value-based-on-the-name-attribute
    product_id = soup_of_page.find_all("div", {"class":"product-brief"})
    index_list = [pid.attrs["data-id"] for pid in product_id]
    price_list = soup_of_page.find_all("div", {"class": "price"})
    seller_list = soup_of_page.select(".product-brief > a")

    ###### attributes to be placed in product csv file ######
    products = {
        "product_index": index_list,
        "product_type": [],
        "product_brand": [],
        "product_name": [],
        "product_price": [],
        "average_rate": [],
        "no_of_rates": [],
        "seller": [],
        "website": []
    }
    
    customers = {
        "product_index": [],
        "username": [],
        "rate_date": [],
        "user_rate": [],
        "buyer_comment": []
    }

    ###### add attributes ######
    for price in price_list:
        products["product_price"].append(handle_price(price.text))

    for seller in seller_list:
        seller = seller.attrs["href"]
        products["website"].append(urllib.parse.unquote(BASE_URL + seller))
        seller = seller.split("/")[1]    
        products["seller"].append(urllib.parse.unquote(seller))
    

    for i, bpn in enumerate(brand_product_name_list):
        s = bpn.text.split(" - ", 1)
        products["product_type"].append(p_type)
        products["product_brand"].append(s[0])
        products["product_name"].append(s[1])
    
    products["average_rate"] = []
    products["no_of_rates"] = []
    for i in range(len(products["website"])):
        while True:
            try:
                print("Getting page " + str(i + 1) + "...")
                print("URL: " + products["website"][i])
                soup = run_asyncio(get_item_page, products["website"][i])
                avg_rating = soup.find("span", {"class": "averageRating"})
                comment_num = soup.find("span", {"class": "comment"})
                products["average_rate"].append(avg_rating.text)
    
                comment_num = int(re.findall(r"\d+", comment_num.text)[0])
                print(str(comment_num) + " comments retrieved from this page with an average rate of " + str(avg_rating.text))
                products["no_of_rates"].append(comment_num)
        
                if comment_num > 0:
                    store_customer_review(products["website"][i], soup, customers)
            
            except:
                print("Error when getting page: ... running again")
                continue
                
            break
    
    products_df = pd.DataFrame(products)
    customers_df = pd.DataFrame(customers)
    return products_df, customers_df

## Run

In [48]:
url = "https://www.hktvmall.com/hktv/zh/search?page=6&q=%3Arelevance%3Acategory%3AAA32300500001%3Azone%3Ahomenfamily%3Astreet%3Amain%3A"
soup = run_asyncio(get_item_page, url)
if not correct_page(soup):
    print("Problem with page source... aborted, please run again")
else:
    p, c = extract_and_store(soup, "laptop")
    p.to_csv("./data/HKTVMall/Products_HKTVMall_laptop7.csv", sep=',', na_rep='N/A', encoding="utf_8_sig")
    c.to_csv("./data/HKTVMall/Customers_HKTVMall_laptop7.csv", sep=',', na_rep='N/A', encoding="utf_8_sig")

Getting page 1...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/LENOVO-Ideapad-330-156-FHD-i78550u-8G-128G1TB-R5302G-WIN10-Home-灰色/p/H6603001_S_IdeaPad_330_15IKB_81DE009DHH
0 comments retrieved from this page with an average rate of 0.0
Getting page 2...
URL: https://www.hktvmall.com/hktv/zh/main/Vertex-恆進/s/S1076001/電子電器/電子電器/電腦及週邊設備/手提電腦/筆記電腦-防盜鋼纜鎖BLACK/p/S1076001_S_4897038340262
0 comments retrieved from this page with an average rate of 0.0
Getting page 3...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/IdeaPad-330S14IKB-14-i78550u-8G-512G-R5352G-WIN10-HOME/p/H6603001_S_IdeaPad_330S_14IKB_81F4018LHH
0 comments retrieved from this page with an average rate of 0.0
Getting page 4...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/IdeaPad-330S14IKB-14-i58250u-8G-512G-SHARE-WIN10-HOME/p/H6603001_S_IdeaPad_330S_14IKB_81F4018JHH
0 comments retrieved from this page with an ave

0 comments retrieved from this page with an average rate of 0.0
Getting page 33...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/A315-156-FHD-i58265u-8G-128G1TB-SHARE-Win10-Home/p/H6603001_S_A315_54_51J1_NXHEFCF003
0 comments retrieved from this page with an average rate of 0.0
Getting page 34...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/A315-156-FHD-i58265u-8G-2TB-MX2302G-Win10-Home-藍色/p/H6603001_S_A315_55G_57JU_NXHG2CF008
Error when getting page: ... running again
Getting page 34...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/A315-156-FHD-i58265u-8G-2TB-MX2302G-Win10-Home-藍色/p/H6603001_S_A315_55G_57JU_NXHG2CF008
Error when getting page: ... running again
Getting page 34...
URL: https://www.hktvmall.com/hktv/zh/main/新域達有限公司/s/H6603001/電子電器/電子電器/電腦及週邊設備/手提電腦/A315-156-FHD-i58265u-8G-2TB-MX2302G-Win10-Home-藍色/p/H6603001_S_A315_55G_57JU_NXHG2CF008
Error when getting p